# A numpy based version of fsl_regfilt
Fsl_regfilt as it stands seems to be very prone to memory exhaustion. In fact it's almost impossible to run it on a multiband aquisition even with about 32GBytes of memory available. This suggests that it is very wasteful of memory so we'll investigate replacing it with an internal numpy based routine.

In [1]:
from __future__ import division, print_function

import numpy as np
from numpy.linalg import pinv
from numpy import inner
import nibabel as nib

### Test files

In [2]:
infile = '../test/refin/filtered_func_data.nii.gz'
mixfile = '../test/refout/melodic.ica/melodic_mix'
compsfile = '../test/refout/classified_motion_ICs.txt'

Input functional series is nii image. When read with nibabel it ends up arranged with time as the *last* index.

In [3]:
func = nib.load(infile).get_data()
nx, ny, nz, nt = func.shape
print('inputdata:', (nx, ny, nz, nt))

inputdata: (64, 64, 34, 180)


The `fsl_regfilt.cc` code generates a mask to remove backgound voxels.

In [4]:
mean = func.mean(axis=3)
min_, max_ = mean.min(), mean.max()
mask = mean > (min_ + (max_ - min_) / 100)

nx, ny, nz = mask.shape
nx, ny, nz
print('mask:', (nx, ny, nz))

mask: (64, 64, 34)


The design matrix we read using `np.loadtxt()` and so it ends up in natural order with the *first* index.

In [5]:
design = np.loadtxt(mixfile)
nt, nc = design.shape
print('design:', (nt, nc))

design: (180, 45)


The `fsl_regfilt.cc` code demeans both the functional data and the design matrix.

In [6]:
mean_r = func.mean(axis=3)
mean_c = design.mean(axis=0)

design_demeaned = design - mean_c
func_demeaned = func - mean_r[..., None]

design_demeaned[0]

array([ 2.45676963,  2.48197556,  2.47422447,  2.39917987,  2.43036357,
        2.474023  ,  2.42873942,  2.42801334,  2.35147706,  2.41762743,
        2.43557958, -3.01345097, -2.69668958, -0.51803066,  2.46157663,
        0.00791765, -2.21859052, -1.72946295, -0.4345678 , -0.91316749,
       -0.64306629, -2.24611874,  0.35066195, -0.73477893, -1.22652366,
       -0.76652087,  0.72942466, -2.67747858, -1.28081546, -0.70359945,
        0.57911162,  1.13386126, -1.10040704,  2.44277085, -0.96658197,
       -0.36108886,  2.4964741 ,  1.99229757, -0.85475222, -0.2094624 ,
       -0.67309535,  0.84547878, -1.58407823, -1.54959604,  0.33945736])

The components are just indices of columns in the design matrix. The list is stored (with offset 1) in a text file,
which we can read with `loadtxt()`.

In [7]:
components = list(np.loadtxt(compsfile, dtype=int, delimiter=',') - 1) 
components

[10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 23, 25, 26, 35, 37, 38, 39, 42, 43]

The actual filtering appears to be performed by the following code (slightly simplified from original):
```c
    Matrix unmixMatrix = pinv(design);
    Matrix maps = unmixMatrix * data;

    Matrix noisedes = design.Column(comps.at(0));
    Matrix noisemaps = maps.Row(comps.at(0)).t();    

    for(int ctr = 1; ctr < (int)comps.size(); ++ctr) {
        noisedes  |= design.Column(comps.at(ctr));
        noisemaps |= maps.Row(comps.at(ctr)).t();
    }

    if(aggressive.value())
        newData = data - noisedes * (pinv(noisedes)*data);
    else
        newData = data - noisedes * noisemaps.t();
        
    newData = newData + ones(newData.Nrows(), 1) * meanR;
```

The `|=` operator seems to have been overloaded to perform horizontal contatenation (ugh).

In [8]:
unmix = pinv(design_demeaned)
print('unmix:', unmix.shape)

maps = inner(unmix, func_demeaned)
nc, nx, ny, nz = maps.shape
print('maps:', (nc, nx, ny, nz))

unmix: (45, 180)
maps: (45, 64, 64, 34)


In [9]:
noisedes = design[:, components]
print('noisedes:', noisedes.shape)

noisemaps = maps[components]
print('noisemaps:', noisemaps.shape)

noisedes: (180, 19)
noisemaps: (19, 64, 64, 34)


In [10]:
new_data_nagg = func_demeaned - inner(noisedes, noisemaps.T).T
new_data_agg  = func_demeaned - inner(noisedes, inner(pinv(noisedes), func_demeaned).T).T

new_data_nagg += mean_r[..., None]
new_data_agg += mean_r[..., None]

In [11]:
data = nib.load(infile).get_data()
design = design = np.loadtxt(mixfile)
components = list(np.loadtxt(compsfile, dtype=int, delimiter=',') - 1) 

design = design - design.mean(axis=0)
nx, ny, nz, nt = data.shape
data = data.reshape((-1, nt))
data_m = data.mean(axis=1)
data = data - data_m[:, None]
unmix = pinv(design)
maps = inner(unmix, data)
noisedes = design[:, components]
noisemaps = maps[components]
noisedes.shape

new_data_nagg = data - inner(noisedes, noisemaps.T).T
new_data_agg  = data - inner(noisedes, inner(pinv(noisedes), data).T).T

new_data_nagg += data_m[..., None]
new_data_agg += data_m[..., None]

filtered_data = data.reshape((nx, ny, nz, nt))

The design matrix is C-contiguous (times, components) as it is read in with `loadtxt()` but the functional data is fortran-contiguous (nx, ny, nz, times). We could transpose the data and collapse the spatial dimensions.  

Then we'd have (nt, nc) and (nt, np). When we take the `pinv()` we get unmix shape of (nc, nt). Ordinary matrix multiplication should work fine then, simplifiying things somewhat.

In [12]:
def ica_filter(data, design, components, aggressive=False, mask=True):
    """Filter functional data by regressing out noise components.
       expects c-contig arrays in natural numpy index ordering
       data: (nt, nz, ny, nz), design: (nt, nc).
       Components is a list of indices (columns) in design.
       We flatten the spatial dimensions of functional data so we
       can treat it as an ordinary matrix.
    """

    nt, nz, ny, nx = data.shape
    ntd, nc = design.shape
    assert ntd == nt
    assert data.flags['C_CONTIGUOUS']
    assert len(components) > 0
    assert 0 <= max(components) < nc 

    # mask out background voxels at less then 1%
    if mask:
        mean_image = data.mean(axis=0)
        min_, max_ = mean_image.min(), mean_image.max()
        mask = mean_image > (min_ + (max_ - min_) / 100)
        data *= mask[None, :]

    # flatten image volumes for ease of manipulation
    data = data.reshape((nt, -1))
    
    # de-mean data and model
    data_means = data.mean(axis=0)
    data = data - data_means
    design = design - design.mean(axis=0)

    # noise components of design
    noise_design = design[:, components]

    if aggressive:
        maps = pinv(noise_design).dot(data)
    else:
        maps = pinv(design).dot(data)[components]

    filtered_data = data - noise_design.dot(maps) + data_means

    # restore shape of image
    return filtered_data.reshape((nt, nz, ny, nx))

Try running directly...

In [13]:
data = nib.load(infile).get_data()
design = np.loadtxt(mixfile)
components = list(np.loadtxt(compsfile, dtype=int, delimiter=',') - 1)
filtered_data = ica_filter(data.T, design, components).T

Wrap up in a function to compare with original using `fsl_regfilt`

In [14]:
def denoising_numpy(infile, outfile, mix, denoise_indices, aggressive=False):
    nii = nib.load(infile)
    data = nii.get_data().T
    # a bit icky but seems to be the standard way to write data back to a nifti image
    nii.get_data()[:] = ica_filter(data, design=mix, components=denoise_indices, aggressive=aggressive).T
    nib.save(nii, outfile)

    
FSLREGFILT = '/usr/local/fsl/bin/fsl_regfilt'
from tempfile import mkstemp
from subprocess import call
import os
def denoising_fsl(infile, outfile, mix, denoise_indices, aggressive=False):
    fd, melmix_file = mkstemp(prefix='denoising', suffix='.txt')
    np.savetxt(melmix_file, mix)

    index_list = ','.join(['%d' % (i+1) for i in denoise_indices])
    regfilt_args = [
        '--in=' + infile, '--design=' + melmix_file,
        '--filter=%s' % index_list,
        '--out=' + outfile
    ]
    if aggressive:
        regfilt_args.append('-a')
    call([FSLREGFILT] + regfilt_args)

    os.unlink(melmix_file)
    os.close(fd)

Compare numpy version with original `fsl_regfilt`, aggressive and nonaggressive versions.

In [15]:
denoising_fsl(infile, outfile='filtered_fsl.nii.gz', mix=np.loadtxt(mixfile), denoise_indices=components)
denoising_numpy(infile, outfile='filtered_numpy.nii.gz', mix=np.loadtxt(mixfile), denoise_indices=components)

res_fsl = nib.load('filtered_fsl.nii.gz').get_data()
res_numpy = nib.load('filtered_numpy.nii.gz').get_data()

assert np.allclose(res_fsl, res_numpy, rtol=1e-06, atol=1e-03)

denoising_fsl(infile, outfile='filtered_agg_fsl.nii.gz', mix=np.loadtxt(mixfile), denoise_indices=components, aggressive=True)
denoising_numpy(infile, outfile='filtered_agg_numpy.nii.gz', mix=np.loadtxt(mixfile), denoise_indices=components, aggressive=True)

res_fsl = nib.load('filtered_agg_fsl.nii.gz').get_data()
res_numpy = nib.load('filtered_agg_numpy.nii.gz').get_data()

assert np.allclose(res_fsl, res_numpy, rtol=1e-06, atol=1e-03)


Timings...

In [16]:
%%timeit
denoising_fsl(infile, outfile='filtered_fsl.nii.gz', mix=np.loadtxt(mixfile), denoise_indices=components)

1 loop, best of 3: 14.2 s per loop


In [17]:
%%timeit
denoising_numpy(infile, outfile='filtered_numpy.nii.gz', mix=np.loadtxt(mixfile), denoise_indices=components)

1 loop, best of 3: 7.24 s per loop


Looks like we get a factor two or three. We should still check the memory usage as this becomes a problem with a large number of components. We probably don't gain anything here by using numpy - the problem is the very large matrix multiplication. We'll probably need to find some out of core / batched way of doing the multiplication. For very large datasets eg from multiband acquisitions we may have problems fitting even the original data set into memory.

In [18]:
!rm -f filtered_*fsl.nii.gz filtered_*numpy.nii.gz